In [165]:
import networkx as nx
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sys
import math
import pickle
import jsonpickle
import csv

In [111]:
n_tiles = pd.read_csv('../qgis/hexagons_centers.csv').shape[0]
n_tiles

1590

In [2]:
stops_with_ids = pd.read_csv("../qgis/stops_with_tile_ids.csv", usecols = ['ID','stop_id'])
stops_with_ids = stops_with_ids.set_index('stop_id')
print(stops_with_ids.shape)
stops_with_ids.head()

(24499, 1)


,ID
stop_id,
132,553
133,949
134,892
135,603
136,572


# Computing centers of hexagons

## Graph 
For complete set of bus and train stops

In [3]:
G = nx.read_gpickle('../graph/complete.graph')

In [4]:
stops_with_ids['degree'] = 0

For the hexagons where no affluence data is available from official **SBB/CFF** sources, we use the max node-degree in the cell as affluence.

### Strategy - choose node with min-average time in tile
For each tile get node with minimum average time to other nodes. 

`for tile:
    min_time = inf.
    matrix = distance(affluence_nodes + nodes ` $\in$ ` group)
    min_time = min(avg_time(matrix), min_time)`

Get all nodes belonging to both known stop_ids in grid and the directed graph.

In [218]:
degrees = {}
for n in G.nodes():
    degrees[n] = nx.degree(G, n)
    
degrees[8501120]

51

In [161]:
gb = stops_with_ids.groupby('ID') # groupby ID of tile
nodes = G.nodes()
affluence_ids = list(affluence.stop_id)
affluence_ids = [e for e in affluence_ids if e in nodes] # keep only nodes in graph
centers_by_tile = {} # map from tile ID to node with minimum average time
i = 0

max_time = 500.0 # provisional measure
for k, gp in gb:
    if(i%50 == 0):
        print("Tile {}/{}".format(i, len(gb)))
    i = i+1
    
    gp_ids = list(gp.index)
    gp_ids = [e for e in gp_ids if e in nodes]
    total_ids = []
    total_ids.extend(affluence_ids)
    total_ids.extend(gp_ids)
    gp_dist = {}
    for s in gp_ids:
        s_dist = []
        for t in total_ids:
            try:
                dist = nx.shortest_path_length(G, s, t)
            except:
                dist = max_time
            s_dist.append(dist)
        s_avg = sum(s_dist) / len(s_dist)
        gp_dist[s] = s_avg
    centers_by_tile[k] = min(gp_dist, key=gp_dist.get) if gp_dist else 0
        

Tile 0/1259
Tile 1/1259
Tile 2/1259
Tile 3/1259
Tile 4/1259
Tile 5/1259
Tile 6/1259
Tile 7/1259
Tile 8/1259
Tile 9/1259
Tile 10/1259
Tile 11/1259
Tile 12/1259
Tile 13/1259
Tile 14/1259
Tile 15/1259
Tile 16/1259
Tile 17/1259
Tile 18/1259
Tile 19/1259
Tile 20/1259
Tile 21/1259
Tile 22/1259
Tile 23/1259
Tile 24/1259
Tile 25/1259
Tile 26/1259
Tile 27/1259
Tile 28/1259
Tile 29/1259
Tile 30/1259
Tile 31/1259
Tile 32/1259
Tile 33/1259
Tile 34/1259
Tile 35/1259
Tile 36/1259
Tile 37/1259
Tile 38/1259
Tile 39/1259
Tile 40/1259
Tile 41/1259
Tile 42/1259
Tile 43/1259
Tile 44/1259
Tile 45/1259
Tile 46/1259
Tile 47/1259
Tile 48/1259
Tile 49/1259
Tile 50/1259
Tile 51/1259
Tile 52/1259
Tile 53/1259
Tile 54/1259
Tile 55/1259
Tile 56/1259
Tile 57/1259
Tile 58/1259
Tile 59/1259
Tile 60/1259
Tile 61/1259
Tile 62/1259
Tile 63/1259
Tile 64/1259
Tile 65/1259
Tile 66/1259
Tile 67/1259
Tile 68/1259
Tile 69/1259
Tile 70/1259
Tile 71/1259
Tile 72/1259
Tile 73/1259
Tile 74/1259
Tile 75/1259
Tile 76/1259
Tile 77/1

In [173]:
df_centers = pd.DataFrame(list(centers_by_tile.items()))
df_centers.columns = ['ID', 'center']

In [178]:
df_centers.to_csv('../centers_by_tile.csv')

In [205]:
df_centers.shape

(1259, 2)

In [204]:
df_centers[df_centers.center == 0].shape

(29, 2)

In [209]:
df_centers[df_centers.ID == 12]

,ID,center
9,12,8501008


### 2nd strategy - max degree of node in tile

In [68]:
nodes = G.nodes()

errors = 0
for n in nodes:
    try:
        stops_with_ids.loc[n].degree = G.degree(n) 
    except KeyError:
        errors += 1
        
print("Nodes missing in graph: {}/{}, when matched with `stops_with_ids`".format(errors, len(nodes)))

Nodes missing in graph: 276/22056, when matched with `stops_with_ids`


### NEXT

Groupby the hexagon ID and aggregate by max affluence.

In [36]:
grouped_by_degree = stops_with_ids.reset_index().groupby('ID')['stop_id', 'degree'].agg({'degree': max})
grouped_by_degree.columns = grouped_by_degree.columns.droplevel()
# grouped_by_degree = grouped_by_degree[grouped_by_degree.degree != 0]
grouped_by_degree.head()

,stop_id,degree
ID,,
2,8595012,0
3,8595953,0
4,8595951,0
5,8595553,0
6,8595952,0


In [37]:
grouped_by_degree[grouped_by_degree.degree != 0].shape[0]

0

In [38]:
grouped_by_degree[grouped_by_degree.stop_id == 8530043]

,stop_id,degree
ID,,
255,8530043,0


In [39]:
missing = grouped_by_degree[grouped_by_degree.degree == 0].shape[0]
total = grouped_by_degree.shape[0]
print("After grouping, missing affluence data for {}/{} cells".format(missing, total))

After grouping, missing affluence data for 1259/1259 cells


## Affluence for available nodes

In [40]:
affluence = pd.read_csv("../gtfs/passagierfrequenz.csv", sep=';')
affluence['x'], affluence['y'] = affluence.geopos.str.split(',', 1).str
affluence = affluence.drop(['Bahnhof_Haltestelle', 'DWV', 'Bemerkungen', 'lod', 'geopos', 'Eigner', 'Bezugsjahr'], axis=1)
affluence.columns = ['Code', 'Affluence', 'x', 'y']
affluence['stop_id'] = 0

In [41]:
affluence.head()

,Code,Affluence,x,y,stop_id
0,AAT,770,47.3359563788,8.76561022548,0
1,AE,2100,47.4677359724,7.60305476436,0
2,ALL,3000,46.4757395273,6.39970400408,0
3,AW,3000,47.5504471564,9.30221759023,0
4,ARN,210,47.4420020412,9.25200601938,0


In [42]:
affluence.to_csv('../gtfs/affluence_code.csv')

In [83]:
affluence_with_ids = pd.read_csv("../qgis/affluence_with_tile_ids.csv")

In [44]:
joined_aff = affluence.join(affluence_with_ids, rsuffix='_r').drop(
        ['x', 'y', 'Code_r', 'stop_id', 'Code'], axis=1)
grouped_by_affluence = joined_aff.groupby('ID').agg(max).reset_index()

print("Centers with affluence data: {}".format(grouped_by_affluence.shape[0]))

Centers with affluence data: 371


## Combine results
Is there relation between degree of node and affluence?


In [230]:
affluence[affluence.stop_id == 8504201]

,Code,Affluence,x,y,stop_id,ID
459,GRS,50,46.80627,6.641937,8504201,96.0


In [220]:
affluence[affluence.Code == 'FRI']

,Code,Affluence,x,y,stop_id,ID
47,FRI,21600,46.803267,7.151023,8504100,297.0


In [196]:
affluence[affluence.Code == 'ZUE']

,Code,Affluence,x,y,stop_id,ID
603,ZUE,396300,47.378177,8.540192,8503000,939.0


In [179]:
affluence[affluence.ID == 939]

,Code,Affluence,x,y,stop_id,ID
601,ZK,1100,47.337332,8.569718,8503100,939.0
603,ZUE,396300,47.378177,8.540192,8503000,939.0
605,ZSTH,68100,47.366611,8.548466,8503003,939.0
606,ZTB,4200,47.350124,8.561372,8503004,939.0
607,ZWIE,7700,47.371472,8.523463,8503011,939.0
640,ZEN,13000,47.364098,8.530806,8503010,939.0
673,ZWOL,2400,47.347440,8.533588,8503009,939.0


In [45]:
affluence = affluence.join(affluence_with_ids, rsuffix='_r').drop(['Code_r'], axis=1)
affluence.head()

,Code,Affluence,x,y,stop_id,ID
0,AAT,770,47.3359563788,8.76561022548,0,1039.0
1,AE,2100,47.4677359724,7.60305476436,0,525.0
2,ALL,3000,46.4757395273,6.39970400408,0,50.0
3,AW,3000,47.5504471564,9.30221759023,0,1337.0
4,ARN,210,47.4420020412,9.25200601938,0,1309.0


In [46]:
stops = pd.read_csv('../gtfs/stops.txt').drop(['Unnamed: 0', 'platform_code'], axis=1)

In [47]:
stops.head()

,stop_id,stop_lon,stop_lat
0,132,7.689360,47.196374
1,133,8.603653,46.154371
2,134,8.435913,46.538322
3,135,7.773846,46.356888
4,136,7.717215,46.433756


### Get closest stop match by euclidean distance between coordinates -> heuristic 
Result stored in `gtfs/affluence_with_stopid.csv`

**WARNING**: takes some time to compute

In [ ]:
MAX_SIZE = sys.maxsize
for i in range(affluence.shape[0]):
    min_ = MAX_SIZE
    min_id = None
    x1 = float(affluence.loc[i].x)
    y1 = float(affluence.loc[i].y)
    for j in range(stops.shape[0]):
        x2 = float(stops.loc[j].stop_lat)
        y2 = float(stops.loc[j].stop_lon)
        dist = math.sqrt(pow(abs(x1-x2),2) + pow(abs(y1 - y2),2))
        if dist < min_:
            min_ = dist
            min_id = stops.loc[j].stop_id
    affluence.set_value(i, 'stop_id', min_id)
    if (i%25 == 0):
        print("{}/{}".format(i, affluence.shape[0]))
        
affluence.to_csv('gtfs/affluence_with_stopid.csv')    

**Instead** use pre-computed file below

In [49]:
affluence = pd.read_csv('../gtfs/affluence_with_stopid.csv')
affluence.drop(['Unnamed: 0'], axis=1, inplace=True)
affluence.head()

,Code,Affluence,x,y,stop_id,ID
0,AAT,770,47.335956,8.765610,8503124,1039.0
1,AE,2100,47.467736,7.603055,8500117,525.0
2,ALL,3000,46.475740,6.399704,8501035,50.0
3,AW,3000,47.550447,9.302218,8506109,1337.0
4,ARN,210,47.442002,9.252006,8506211,1309.0


Merge max affluence nodes for each cell with corresponding stop_id (gtfs format).

In [50]:
affluence_stop_ids = grouped_by_affluence.merge(affluence, left_on=['ID', 'Affluence'],
                          right_on=['ID', 'Affluence'])[['ID', 'stop_id']]

In [51]:
affluence_stop_ids.head()

,ID,stop_id
0,3.0,8501001
1,6.0,8501003
2,11.0,8516155
3,12.0,8501008
4,22.0,8501022


Combine the results from affluence and degree to one collection

In [52]:
cell_centers = grouped_by_degree.drop(['degree'], axis=1)
for i in range(affluence_stop_ids.shape[0]):
    cell_id = affluence_stop_ids.loc[i].ID
    stop_id = affluence_stop_ids.loc[i].stop_id
    cell_centers.set_value(cell_id, 'stop_id', stop_id)
    

Get list of all centers

## IMPORTANT: sort the centers by tile-id

In [118]:
cell_centers = cell_centers.reindex(range(1, n_tiles+1), fill_value=0)

In [119]:
lst_centers = list(map(lambda x: int(x), cell_centers.stop_id))

In [126]:
nodes = G.nodes()
filt_centers = []
for c in lst_centers:
    if c in nodes:
        filt_centers.append(c)
    else: 
        filt_centers.append(0)

In [129]:
with open('../res/center_nodes', 'wb') as f:
    pickle.dump(filt_centers, f)